In [1]:
import numpy as np

For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

We can implement this in Python. First we define the IRR functions.

In [2]:
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative


For CMS rate payment, since $g(F)=F^\frac{1}{p} - 0.04^\frac{1}{q}, p = 4, q= 2$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K^\frac{1}{4} - 0.04^\frac{1}{2} \\
g'(K) &= \frac{1}{4}K^{-\frac{3}{4}} \\
g''(K) &= -\frac{3}{16}K^{-\frac{7}{4}}
\end{split}
\end{equation*}

In [3]:
def g_0(K):
    return K**(1/4) - 0.04**(1/2)

def g_1(K):
    return (1/4) * K**(-3/4)

def g_2(K):
    return (-3/16) * K**(-7/4)


In [5]:
def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative

# NOT DONE

We will also need to implement the IRR-settled payer and receiver swaption formulae:

  \begin{equation*}
    \begin{split}
      V^{pay}_{n,N}(0) &= D(0,T_n) \cdot \text{IRR}(S_{n,N}(0)) \cdot \text{Black76Call}(S_{n,N}(0),K,\sigma_{n,N},T) \\
      V^{rec}_{n,N}(0) &= D(0,T_n) \cdot \text{IRR}(S_{n,N}(0)) \cdot \text{Black76Put}(S_{n,N}(0),K,\sigma_{n,N},T) \\
    \end{split}
  \end{equation*}

where $S_{n,N}(0)=F$ is today's forward swap rate calculated based on the curves we bootstrapped, and $\sigma_{n,N}$ is the SABR implied volatility calibrated to swaption market data.

In [7]:
from scipy.stats import norm

def Black76Call(F, K, sigma, T):
    d1 = (np.log(F/K) + (sigma**2)*T/2)/(sigma*np.sqrt(T))
    d2 = (np.log(F/K) - (sigma**2)*T/2)/(sigma*np.sqrt(T))
    return (F*norm.cdf(d1) - K*norm.cdf(d2))

def Black76Put(F, K, sigma, T):
    d1 = (np.log(F/K) + (sigma**2)*T/2)/(sigma*np.sqrt(T))
    d2 = (np.log(F/K) - (sigma**2)*T/2)/(sigma*np.sqrt(T))
    return (K*norm.cdf(-d2) - F*norm.cdf(-d1))

In [8]:
def payer_swaption(discount, S_n_N, K, sigma, T, m , N):
    return discount * IRR_0(S_n_N, m, N) * Black76Call(S_n_N, K, sigma, T)

def receiver_swaption(discount, S_n_N, K, sigma, T, m , N):
    return discount * IRR_0(S_n_N, m, N) * Black76Put(S_n_N, K, sigma, T)

  \begin{equation*}
    \begin{split}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

# Part 1

In [ ]:
from scipy.integrate import quad

def call_integrand(discount, S_n_N, K, sigma, T, m , N):
    price = h_2(K, m, N) * payer_swaption(discount, S_n_N, K, sigma, T, m , N)
    return price

def put_integrand(discount, S_n_N, K, sigma, T, m , N):
    price = h_2(K, m, N) * receiver_swaption(discount, S_n_N, K, sigma, T, m , N)
    return price


I_put = quad(lambda x: call_integrand(discount, S_n_N, x, sigma, T, m , N), 0.0, F)
I_call = quad(lambda x: put_integrand(discount, S_n_N, x, sigma, T, m , N), F, 5000)

v_0 = discount * g_0(F)\
     + h_1(F)*(payer_swaption(discount, S_n_N, F, sigma, T, m , N) - receiver_swaption(discount, S_n_N, F, sigma, T, m , N))\
     + I_put \
     + I_call